# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-06-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
pa = xr.open_dataset('F:/Data/Project_anqing/May/COMBINE_ACONC_v54_ISAM_intel_CN3AH_135X138_2023_pa.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range('2023-05-01-00','2023-05-15-23',freq='h')
times

DatetimeIndex(['2023-05-01 00:00:00', '2023-05-01 01:00:00',
               '2023-05-01 02:00:00', '2023-05-01 03:00:00',
               '2023-05-01 04:00:00', '2023-05-01 05:00:00',
               '2023-05-01 06:00:00', '2023-05-01 07:00:00',
               '2023-05-01 08:00:00', '2023-05-01 09:00:00',
               ...
               '2023-05-15 14:00:00', '2023-05-15 15:00:00',
               '2023-05-15 16:00:00', '2023-05-15 17:00:00',
               '2023-05-15 18:00:00', '2023-05-15 19:00:00',
               '2023-05-15 20:00:00', '2023-05-15 21:00:00',
               '2023-05-15 22:00:00', '2023-05-15 23:00:00'],
              dtype='datetime64[ns]', length=360, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [6]:
days=1 # set spin-up days
dataset1=xr.Dataset(
    data_vars=dict(
        # ! vars from PA
        HADV_O3=(['time','level','y','x'],pa.HADV_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Horizontal Advection of O3','units':'ug m-3'}),
        ZADV_O3=(['time','level','y','x'],pa.ZADV_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Veritical Advection of O3','units':'ug m-3'}),
        HDIF_O3=(['time','level','y','x'],pa.HDIF_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Horizontal Diffusion of O3','units':'ug m-3'}),
        VDIF_O3=(['time','level','y','x'],pa.VDIF_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Vertical Diffusion of O3','units':'ug m-3'}),
        DDEP_O3=(['time','level','y','x'],pa.DDEP_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Dry Deposition of O3','units':'ug m-3'}),
        CHEM_O3=(['time','level','y','x'],pa.CHEM_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Gas-phase Chemistry of O3','units':'ug m-3'}),

        HNO3prod=(['time','level','y','x'],pa.HNO3prod[days*24-8:-8,:,:,:].data,{'long name':'Production Velocity of HNO3','units':'ppbv'}),
        H2O2prod=(['time','level','y','x'],pa.H2O2prod[days*24-8:-8,:,:,:].data,{'long name':'Production Velocity of H2O2','units':'ppbv'}),
        O3prod=(['time','level','y','x'],pa.O3prod[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Production Velocity of O3','units':'ug m-3'}),
        O3loss=(['time','level','y','x'],pa.O3loss[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Loss Velocity of O3','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset1

<xarray.Dataset>
Dimensions:    (time: 360, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-01 ... 2023-05-15T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    HADV_O3    (time, level, y, x) float32 178.1 -78.19 -769.9 ... 59.54 150.0
    ZADV_O3    (time, level, y, x) float32 -185.3 106.5 794.5 ... -27.74 -120.8
    HDIF_O3    (time, level, y, x) float32 0.3081 0.6287 ... -2.026 -2.029
    VDIF_O3    (time, level, y, x) float32 31.27 5.205 ... -0.0612 -0.05871
    DDEP_O3    (time, level, y, x) float32 -23.33 -34.23 -20.48 ... 0.0 0.0 0.0
    CHEM_O3    (time, level, y, x) float32 -0.9225 -0.5972 -0.3379 ... 0.0 0.0
    HNO3prod   (time, level, y, x) float32 0.0183 0.01233 ... 1.612e-05
    H2O2prod   (time, level, y, x) float32 0.001291 0.001189 ... 1.032e-08
    O3prod     (time, level, y, x) float32 0.000277 0.0003434 ... 1.59e-11
    O3loss     (time, level, y, x) float32 0.9228 0.5975 ... 0.001302 0.001304
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset1.data_vars}
dataset1.to_netcdf('D:/Download/May_PA_O3-1.nc',encoding=encoding)

In [8]:
days=1 # set spin-up days
dataset2=xr.Dataset(
    data_vars=dict(
        # ! vars from PA
        HADV_NO=(['time','level','y','x'],pa.HADV_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Horizontal Advection of NO','units':'ug m-3'}),
        ZADV_NO=(['time','level','y','x'],pa.ZADV_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Veritical Advection of NO','units':'ug m-3'}),
        HDIF_NO=(['time','level','y','x'],pa.HDIF_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Horizontal Diffusion of NO','units':'ug m-3'}),
        VDIF_NO=(['time','level','y','x'],pa.VDIF_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Vertical Diffusion of NO','units':'ug m-3'}),
        DDEP_NO=(['time','level','y','x'],pa.DDEP_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Dry Deposition of NO','units':'ug m-3'}),
        CHEM_NO=(['time','level','y','x'],pa.CHEM_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Gas-phase Chemistry of NO','units':'ug m-3'}),
        
        HADV_NO2=(['time','level','y','x'],pa.HADV_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Horizontal Advection of NO2','units':'ug m-3'}),
        ZADV_NO2=(['time','level','y','x'],pa.ZADV_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Veritical Advection of NO2','units':'ug m-3'}),
        HDIF_NO2=(['time','level','y','x'],pa.HDIF_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Horizontal Diffusion of NO2','units':'ug m-3'}),
        VDIF_NO2=(['time','level','y','x'],pa.VDIF_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Vertical Diffusion of NO2','units':'ug m-3'}),
        DDEP_NO2=(['time','level','y','x'],pa.DDEP_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Dry Deposition of NO2','units':'ug m-3'}),
        CHEM_NO2=(['time','level','y','x'],pa.CHEM_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Gas-phase Chemistry of NO2','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset2

<xarray.Dataset>
Dimensions:    (time: 360, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-01 ... 2023-05-15T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    HADV_NO    (time, level, y, x) float32 -0.06417 -0.09705 ... 2.091e-10
    ZADV_NO    (time, level, y, x) float32 -0.04872 0.004192 ... -2.454e-10
    HDIF_NO    (time, level, y, x) float32 0.0009171 0.001678 ... 3.802e-12
    VDIF_NO    (time, level, y, x) float32 -0.09756 -0.004949 ... 7.237e-14
    DDEP_NO    (time, level, y, x) float32 -0.001188 -0.0006881 ... 0.0 0.0
    CHEM_NO    (time, level, y, x) float32 -0.4333 -0.2167 ... 1.072e-10
    HADV_NO2   (time, level, y, x) float32 2.193 -0.7001 ... 0.0001388 0.00779
    ZADV_NO2   (time, level, y, x) float32 -2.197 0.7316 ... -0.00182 -0.009127
    HDIF_NO2   (time, level, y, x) float32 0.1047 0.02489 ... 0.0001899
    VDIF_NO2   (time, level, y, x) float32 -0.6246 -0.1854 ... 3.336e-06
    DDEP_NO2   (time, level, y, x) float32 -0.09245 -0.07101 ... 0.0 0.0
    CHEM_NO2   (time, level, y, x) float32 0.4963 0.2484 ... -0.001756 -0.001752
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [9]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset2.data_vars}
dataset2.to_netcdf('D:/Download/May_PA_NOx-1.nc',encoding=encoding)